In [29]:
from transformers import VideoMAEForVideoClassification
from transformers import AutoImageProcessor, VideoMAEForPreTraining, VideoMAEConfig
import torch.nn as nn
import torch
import numpy as np


In [36]:
model_ckpt = "MCG-NJU/videomae-base" # pre-trained model from which to fine-tune
model = VideoMAEForVideoClassification.from_pretrained(model_ckpt, attn_implementation="sdpa", torch_dtype=torch.float16)

Some weights of VideoMAEForVideoClassification were not initialized from the model checkpoint at MCG-NJU/videomae-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
print(model.classifier.in_features)
print(model.classifier.out_features)

768
2


In [16]:
config = VideoMAEConfig()
custom_classes_classifier = VideoMAEForVideoClassification(config)
custom_classes_classifier.config

VideoMAEConfig {
  "_attn_implementation_autoset": true,
  "attention_probs_dropout_prob": 0.0,
  "decoder_hidden_size": 384,
  "decoder_intermediate_size": 1536,
  "decoder_num_attention_heads": 6,
  "decoder_num_hidden_layers": 4,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "videomae",
  "norm_pix_loss": true,
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_frames": 16,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "transformers_version": "4.50.0",
  "tubelet_size": 2,
  "use_mean_pooling": true
}

In [ ]:
num_frames = 300
video = list(np.random.randint(0, 256, (num_frames, 3, 224, 224)))

In [31]:
for name, param in model.named_parameters():
    print(name, param.shape)

videomae.embeddings.patch_embeddings.projection.weight torch.Size([768, 3, 2, 16, 16])
videomae.embeddings.patch_embeddings.projection.bias torch.Size([768])
videomae.encoder.layer.0.attention.attention.q_bias torch.Size([768])
videomae.encoder.layer.0.attention.attention.v_bias torch.Size([768])
videomae.encoder.layer.0.attention.attention.query.weight torch.Size([768, 768])
videomae.encoder.layer.0.attention.attention.key.weight torch.Size([768, 768])
videomae.encoder.layer.0.attention.attention.value.weight torch.Size([768, 768])
videomae.encoder.layer.0.attention.output.dense.weight torch.Size([768, 768])
videomae.encoder.layer.0.attention.output.dense.bias torch.Size([768])
videomae.encoder.layer.0.intermediate.dense.weight torch.Size([3072, 768])
videomae.encoder.layer.0.intermediate.dense.bias torch.Size([3072])
videomae.encoder.layer.0.output.dense.weight torch.Size([768, 3072])
videomae.encoder.layer.0.output.dense.bias torch.Size([768])
videomae.encoder.layer.0.layernorm_befo

In [30]:
model.classifier = nn.Linear(model.classifier.in_features, 10)
#random initialize weights
nn.init.xavier_uniform_(model.classifier.weight)

Parameter containing:
tensor([[ 2.9160e-02, -5.9799e-02, -6.5543e-02,  ..., -5.2898e-02,
         -5.6438e-02,  2.7119e-02],
        [ 7.8855e-02,  4.2638e-02, -7.4459e-02,  ..., -1.0165e-02,
         -7.1323e-02,  2.3759e-02],
        [-3.7065e-02,  6.4580e-02,  4.2882e-03,  ...,  3.5372e-02,
         -7.2661e-02,  2.8763e-03],
        ...,
        [ 2.2121e-05, -3.2489e-02,  4.4185e-02,  ...,  6.5742e-02,
         -3.4614e-02,  6.6461e-02],
        [-7.5132e-02, -1.3558e-02,  4.2514e-02,  ...,  4.4074e-02,
         -2.4874e-02, -7.3005e-04],
        [ 3.8914e-02, -2.1927e-02, -7.5983e-02,  ..., -3.0596e-02,
          8.1747e-02, -4.6111e-02]], requires_grad=True)

In [20]:
image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
pixel_values = image_processor(video, return_tensors="pt", use_fast = True).pixel_values


/home/jth264/.conda/envs/benchmark/lib/python3.12/site-packages/transformers/image_processing_utils.py:42: UserWarning: The following named arguments are not valid for `VideoMAEImageProcessor.preprocess` and were ignored: 'use_fast'
  return self.preprocess(images, **kwargs)


In [21]:
type(pixel_values)

torch.Tensor

In [22]:
pixel_values.shape

torch.Size([1, 16, 3, 224, 224])

In [23]:
outputs = custom_classes_classifier(pixel_values)

In [25]:
type(outputs)

transformers.modeling_outputs.ImageClassifierOutput

In [26]:
outputs

ImageClassifierOutput(loss=None, logits=tensor([[-0.1453,  0.1725]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
import av
import torch
import numpy as np

from transformers import AutoImageProcessor, VideoMAEForVideoClassification
from huggingface_hub import hf_hub_download

np.random.seed(0)


def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    '''
    Sample a given number of frame indices from the video.
    Args:
        clip_len (`int`): Total number of frames to sample.
        frame_sample_rate (`int`): Sample every n-th frame.
        seg_len (`int`): Maximum allowed index of sample's last frame.
    Returns:
        indices (`List[int]`): List of sampled frame indices
    '''
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices


# video clip consists of 300 frames (10 seconds at 30 FPS)
file_path = hf_hub_download(
    repo_id="nielsr/video-demo", filename="eating_spaghetti.mp4", repo_type="dataset"
)
container = av.open(file_path)

# sample 16 frames
indices = sample_frame_indices(clip_len=16, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
video = read_video_pyav(container, indices)

image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base-finetuned-kinetics")
model = VideoMAEForVideoClassification.from_pretrained("MCG-NJU/videomae-base-finetuned-kinetics")

inputs = image_processor(list(video), return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# model predicts one of the 400 Kinetics-400 classes
predicted_label = logits.argmax(-1).item()
print(model.config.id2label[predicted_label])

In [39]:
from huggingface_hub import hf_hub_download

hf_dataset_identifier = "sayakpaul/ucf101-subset"
filename = "UCF101_subset.tar.gz"
destination_path = "/share/j_sun/jth264"  # Change this to your desired directory

file_path = hf_hub_download(
    repo_id=hf_dataset_identifier, 
    filename=filename, 
    repo_type="dataset",
    local_dir=destination_path  # Specify the download directory
)
